# `formulas`: An Excel formulas interpreter in Python.

## Parsing formula
An example how to parse and execute an Excel formula is the following:

In [ ]:
import formulas
func = formulas.Parser().ast('=(1 + 1) + B3 / A2')[1].compile()

To visualize formula model and get the input order you can do the following:

In [ ]:
list(func.inputs)

In [ ]:
func.plot(view=False)

Finally to execute the formula and plot the workflow:

In [ ]:
func(1, 5)

In [ ]:
func.plot(workflow=True, view=False)

Excel workbook
--------------
An example how to load, calculate, and write an Excel workbook is the following:

In [ ]:
xl_model = formulas.ExcelModel().loads("../test/test_files/excel.xlsx").finish()
xl_model.calculate()

In [ ]:
xl_model.write()

> If you have or could have **circular references**, add `circular=True`
   to `finish` method.

To plot the dependency graph that depict relationships between Excel cells:

In [ ]:
dsp = xl_model.dsp
dsp.plot(view=False)

To overwrite the default inputs that are defined by the excel file or to impose
some value to a specific cell:

In [ ]:
xl_model.calculate(
    inputs={
        "'[EXCEL.XLSX]DATA'!A2": 3,  # To overwrite the default value.
        "'[EXCEL.XLSX]DATA'!B3": 1  # To impose a value to B3 cell.
    },
    outputs=[
       "'[EXCEL.XLSX]DATA'!C2", "'[EXCEL.XLSX]DATA'!C4"
    ] # To define the outputs that you want to calculate.
)

To build a single function out of an excel model with fixed inputs and outputs,
you can use the `compile` method of the `ExcelModel` that returns a
[`DispatchPipe`](https://schedula.readthedocs.io/en/master/_build/schedula/utils/dsp/schedula.utils.dsp.DispatchPipe.html#schedula.utils.dsp.DispatchPipe). This is a function where the inputs and outputs are defined by
the data node ids (i.e., cell references).

In [ ]:
func = xl_model.compile(
    inputs=[
        "'[EXCEL.XLSX]DATA'!A2",  # First argument of the function.
        "'[EXCEL.XLSX]DATA'!B3"   # Second argument of the function.
    ], # To define function inputs.
    outputs=[
        "'[EXCEL.XLSX]DATA'!C2", "'[EXCEL.XLSX]DATA'!C4"
    ] # To define function outputs.
)
func

In [ ]:
[v.value[0, 0] for v in func(3, 1)]  # To retrieve the data.

In [ ]:
func.plot(view=False)  # Set view=True to plot in the default browser.

Custom functions
----------------
An example how to add a custom function to the formula parser is the following:

In [ ]:
import formulas
FUNCTIONS = formulas.get_functions()
FUNCTIONS['MYFUNC'] = lambda x, y: 1 + y + x
func = formulas.Parser().ast('=MYFUNC(1, 2)')[1].compile()
func()